In [13]:
import requests
import json
import pandas as pd
df = pd.read_csv('pre_clean_data.csv')
df

,name,price,address,Type,nbeds,nbath,laundry,heating,parking,air_conditioner,furnished,animal_friendly,Area
0,3 beds 3 baths townhouse,3300,"kelowna, bc, v1y 5t5",Townhouse,3,3,in-unit laundry,gas heating,garage parking,central ac,unfurnished,No pets,1371
1,2 beds 1 bath house,2000,"west kelowna, bc, v4t",House,2,1,in-unit laundry,Not specified,Not specified,Not specified,Not specified,dog and cat friendly,914
2,1 bed 1 bath apartment,1250,"kelowna, bc, v1v",Apartment/Condo,1,1,laundry in building,Not specified,parking available,Not specified,unfurnished,No pets,910
3,1 bedroom available in 2 bedroom basement suite,650,"west kelowna, bc",Room,1,1,in-unit laundry,central heating,street parking,Not specified,unfurnished,No pets,150
4,3 beds 1 bath house,2000,"kelowna, bc, v1v 1r1",House,3,1,Not specified,Not specified,Not specified,Not specified,Not specified,No pets,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,one bedroom in 3 bed 3 bath condo,1000,"kelowna, bc, v1v 0a5",Apartment/Condo,3,3,Not specified,Not specified,Not specified,Not specified,unfurnished,No pets,910
1264,come stay with us pet friendly kelowna cottage,1111,"central okanagan, bc, v1z 3r8",House,3,2,Not specified,Not specified,Not specified,Not specified,unfurnished,dog friendly,1200
1265,room for rent available in rutland,850,"kelowna, bc",Room,1,1,Not specified,Not specified,Not specified,Not specified,Not specified,No pets,150
1266,room for rent for girls in rutland dm for more...,900,"kelowna, bc",Room,1,1,Not specified,Not specified,Not specified,Not specified,unfurnished,No pets,150


In [8]:
df_address = df['address'].drop_duplicates()
df_address = pd.DataFrame(df_address)

In [5]:
for index, row in df_address.iterrows():
    api_address = row['address']
    parameters = {
        "key": "API_KEY",
        "location": api_address
    }
    response = requests.get("http://www.mapquestapi.com/geocoding/v1/address", params=parameters)
    data = response.json()['results']
    lat = data[0]['locations'][0]['latLng']['lat']
    lng = data[0]['locations'][0]['latLng']['lng']
    df_address.at[index, 'lat'] = lat
    df_address.at[index, 'lng'] = lng

In [14]:
df_address.to_csv("../../processed/location.csv", index = False)

Now I need to merge it.

In [15]:
df_address = pd.read_csv('../../processed/location.csv')

In [17]:
df = df.merge(df_address, on = 'address')

In [18]:
import folium
map1 = folium.Map(
    location=[49.9138315, -119.439960],
    tiles='cartodbpositron',
    zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lng"]]).add_to(map1), axis=1)
map1

Unfortunately, Github can't display the map, so I decided to paste the image here.

![Kelowna Map!](../../../images/map.png)

In [19]:
df.to_csv("../../processed/clean_kelowna_housing_data.csv", index = False)
df.to_excel('../../processed/clean_kelowna_housing_data.xlsx')